# Clean & Analyze Social Media

## Introduction

Social media has become a ubiquitous part of modern life, with platforms such as Instagram, Twitter, and Facebook serving as essential communication channels. Social media data sets are vast and complex, making analysis a challenging task for businesses and researchers alike. In this project, we explore a simulated social media, for example Tweets, data set to understand trends in likes across different categories.

## Prerequisites

To follow along with this project, you should have a basic understanding of Python programming and data analysis concepts. In addition, you may want to use the following packages in your Python environment:

- pandas
- Matplotlib

These packages should already be installed in Coursera's Jupyter Notebook environment, however if you'd like to install additional packages that are not included in this environment or are working off platform you can install additional packages using `!pip install packagename` within a notebook cell such as:

- `!pip install pandas`
- `!pip install matplotlib`

## Project Scope

The objective of this project is to analyze tweets (or other social media data) and gain insights into user engagement. We will explore the data set using visualization techniques to understand the distribution of likes across different categories. Finally, we will analyze the data to draw conclusions about the most popular categories and the overall engagement on the platform.

## Step 1: Importing Required Libraries

As the name suggests, the first step is to import all the necessary libraries that will be used in the project. In this case, we need pandas, numpy, matplotlib, seaborn, and random libraries.

Pandas is a library used for data manipulation and analysis. Numpy is a library used for numerical computations. Matplotlib is a library used for data visualization. Seaborn is a library used for statistical data visualization. Random is a library used to generate random numbers.

In [1]:
import csv # Python library used for reading and writing tabular data in CSV format.
import pandas as pd # Python library used for working with data sets. It has functions for analyzing, cleaning, exploring, and manipulating data. 
import json # Python library for encoding and decoding custom objects by using JSON encoder and decoder classes.
import numpy as np # Python library for adding support to large, multi-dimensional arrays and matrices, along with a large collection of high-level mathematical functions to operate on these arrays.
import seaborn as sns #Python library for statistical visualizations
import statistics as stat #Python library for statistical operations
import matplotlib.pyplot as plt # Python library for creating graphs
import random # Python library for generating random data

from scipy import stats # Python library for linear regression

random.seed(4) # Set the seed for generating the same random data.
np.random.seed(4)
total = 1000

In [2]:
# Define various lists for data generation

categories = ["Food", "Travel", "Fashion", "Fitness", "Music", "Culture", "Family", "Health"]
content = ["Image", "Video", "Story"]
objectives = ["Increase Engagement", "Brand Awareness", "Conversions", "Lead Generation"]
days = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
time = ["Morning", "Afternoon", "Evening", "Night"]
sentiment = ["Positive", "Neutral", "Negative"]
campaigns = ["Campaign 1", "Campaign 2", "Campaign 3", "Campaign 4", "Campaign 5"]
platforms = ["Instagram", "Twitter-X", "Facebook", "LinkedIn"]
conversions = ["Purchases", # Completing a transaction or buying a product
               "Sign-Up's", # Registering for an account, newsletter, or service
               "Downloads", # Downloading a file, app, or resource
               "Form Submissions", # Filling out and submitting a contact form or survey
               "Lead Generation"] # Providing contact information or expressing interest in a service or product


# Define the Change in Followers
initital_followers = 100000 # Initialize the first follower count
current_followers = initital_followers
# Create Lists to hold follower changes and current followers
change_in_followers = []
follower_counts = []
# Generate change in followers with random increases and decreases
for i in range(total):
    change = np.random.randint(-500, 5000)
    change_in_followers.append(change)
    # Update the current follower count
    current_followers += change
    follower_counts.append(max(current_followers, 0)) # Ensure non-negative follower counts

In [3]:
# Create Random Data for the Project

data = {# Post Information
       "ID":[i for i in range(total)],
       "Date":pd.date_range("2020-01-01", periods = total),
       "Time":[random.choice(time) for i in range(total)],
       "Content":[random.choice(content) for i in range(total)],
       "Category":[random.choice(categories) for i in range(total)],
       "Platform":[random.choice(platforms) for i in range(total)],
    
       # Post Metrics
       "Likes":np.random.randint(0, 10000, size = total),
       "Comments":np.random.randint(0, 500, size = total),
       "Shares":np.random.randint(0,2000, size = total),
       "Views":np.random.randint(1000, 50000, size = total),
       "Hashtags":np.random.randint(0, 15, size = total),
        
       # Audience Metrics
       "Followers": follower_counts,
       "Change in Followers":change_in_followers,
        
       # Business Metrics
       "Campaign ID": [random.choice(campaigns) for i in range(total)],
       "Campaign Budget":np.random.randint(1000, 50000, size = total),
       "Post Objective": [random.choice(objectives) for i in range(total)],
       "Impressions": np.random.randint(1000, 100000, size = total),
       "Clicks":np.random.randint(0, 5000, size = total),
       "Conversions":np.random.randint(0, 1000, size = total)
        
                       }

In [4]:
# Create a DataFrame from the data dictionary
df = pd.DataFrame(data)
df.head()

,ID,Date,Time,Content,Category,Platform,Likes,Comments,Shares,Views,Hashtags,Followers,Change in Followers,Campaign ID,Campaign Budget,Post Objective,Impressions,Clicks,Conversions
0,0,2020-01-01,Afternoon,Story,Fashion,Twitter-X,7309,121,1237,24804,4,100646,646,Campaign 5,6104,Lead Generation,76100,2574,384
1,1,2020-01-02,Evening,Story,Fashion,Twitter-X,8969,306,1346,40353,13,100320,-326,Campaign 4,9227,Lead Generation,45468,2161,496
2,2,2020-01-03,Morning,Video,Health,Twitter-X,551,308,1338,6342,13,102307,1987,Campaign 5,11827,Increase Engagement,88613,2896,409
3,3,2020-01-04,Night,Video,Fashion,LinkedIn,6394,91,399,41749,11,102516,209,Campaign 4,41304,Increase Engagement,59807,1192,44
4,4,2020-01-05,Night,Video,Music,Twitter-X,7998,163,579,34025,3,105687,3171,Campaign 2,31975,Increase Engagement,16819,2817,423


In [5]:
# Clean the data

# Ensure the funnel follows Impressions -> Clicks -> Conversions
df = df[(df["Impressions"] > df["Clicks"]) & (df["Clicks"] > df["Conversions"])]

# Impressions -> Views
df = df[(df["Impressions"] > df["Views"])]

df.loc[~df["Content"].isin(["Video", "Story"]), "Views"] = 0

In [6]:


df["Day of the Week"] = df["Date"].dt.day_name()
df['Weekday-Weekend'] = df['Date'].dt.day_name().apply(lambda x: 'Weekend' if x in ['Saturday', 'Sunday'] 
                                                       else 'Weekday')

df["Engaement Rate"] = (df["Likes"] + df["Comments"] + df["Shares"]) / df["Impressions"]
df["CTR"] = df["Clicks"] / df["Impressions"]
df["ROI"] = (df["Conversions"] * 100) / df["Campaign Budget"]
# Review the Head of the DataFrame
df.head(10)

,ID,Date,Time,Content,Category,Platform,Likes,Comments,Shares,Views,...,Campaign Budget,Post Objective,Impressions,Clicks,Conversions,Day of the Week,Weekday-Weekend,Engaement Rate,CTR,ROI
0,0,2020-01-01,Afternoon,Story,Fashion,Twitter-X,7309,121,1237,24804,...,6104,Lead Generation,76100,2574,384,Wednesday,Weekday,0.113890,0.033824,6.290957
1,1,2020-01-02,Evening,Story,Fashion,Twitter-X,8969,306,1346,40353,...,9227,Lead Generation,45468,2161,496,Thursday,Weekday,0.233593,0.047528,5.375528
2,2,2020-01-03,Morning,Video,Health,Twitter-X,551,308,1338,6342,...,11827,Increase Engagement,88613,2896,409,Friday,Weekday,0.024793,0.032681,3.458189
3,3,2020-01-04,Night,Video,Fashion,LinkedIn,6394,91,399,41749,...,41304,Increase Engagement,59807,1192,44,Saturday,Weekend,0.115104,0.019931,0.106527
6,6,2020-01-07,Morning,Video,Travel,Instagram,2773,147,1986,31214,...,10546,Lead Generation,38746,1159,850,Tuesday,Weekday,0.126620,0.029913,8.059928
10,10,2020-01-11,Evening,Story,Health,LinkedIn,2361,388,1654,23564,...,31847,Lead Generation,78347,4584,540,Saturday,Weekend,0.056199,0.058509,1.695607
12,12,2020-01-13,Afternoon,Video,Fashion,LinkedIn,8183,233,19,25903,...,44814,Conversions,84974,280,277,Monday,Weekday,0.099266,0.003295,0.618110
13,13,2020-01-14,Evening,Story,Health,Instagram,1836,200,1088,37481,...,12821,Increase Engagement,79028,4942,583,Tuesday,Weekday,0.039530,0.062535,4.547227
14,14,2020-01-15,Evening,Image,Culture,Instagram,3796,290,785,0,...,29654,Conversions,68197,1711,762,Wednesday,Weekday,0.071425,0.025089,2.569636
15,15,2020-01-16,Afternoon,Image,Fashion,Instagram,4342,216,1896,0,...,37311,Increase Engagement,51794,4663,360,Thursday,Weekday,0.124609,0.090030,0.964863


In [7]:
# Review the Head of the DataFrame
df.head()

,ID,Date,Time,Content,Category,Platform,Likes,Comments,Shares,Views,...,Campaign Budget,Post Objective,Impressions,Clicks,Conversions,Day of the Week,Weekday-Weekend,Engaement Rate,CTR,ROI
0,0,2020-01-01,Afternoon,Story,Fashion,Twitter-X,7309,121,1237,24804,...,6104,Lead Generation,76100,2574,384,Wednesday,Weekday,0.113890,0.033824,6.290957
1,1,2020-01-02,Evening,Story,Fashion,Twitter-X,8969,306,1346,40353,...,9227,Lead Generation,45468,2161,496,Thursday,Weekday,0.233593,0.047528,5.375528
2,2,2020-01-03,Morning,Video,Health,Twitter-X,551,308,1338,6342,...,11827,Increase Engagement,88613,2896,409,Friday,Weekday,0.024793,0.032681,3.458189
3,3,2020-01-04,Night,Video,Fashion,LinkedIn,6394,91,399,41749,...,41304,Increase Engagement,59807,1192,44,Saturday,Weekend,0.115104,0.019931,0.106527
6,6,2020-01-07,Morning,Video,Travel,Instagram,2773,147,1986,31214,...,10546,Lead Generation,38746,1159,850,Tuesday,Weekday,0.126620,0.029913,8.059928


In [8]:
# Create a table showing the category counts
category_counts = df.Category.value_counts()
print(category_counts)

Category
Travel     98
Music      90
Culture    88
Food       84
Fitness    84
Fashion    82
Health     79
Family     78
Name: count, dtype: int64


In [9]:
# Describe the numeric values of the DataFrame
df.describe()

,ID,Date,Likes,Comments,Shares,Views,Hashtags,Followers,Change in Followers,Campaign Budget,Impressions,Clicks,Conversions,Engaement Rate,CTR,ROI
count,683.000000,683,683.000000,683.000000,683.000000,683.000000,683.000000,6.830000e+02,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000,683.000000
mean,501.661786,2021-05-16 15:52:58.330893312,4977.648609,238.875549,1014.979502,15970.855051,7.029283,1.251880e+06,2295.200586,25576.894583,60896.038067,2759.841874,472.754026,0.130568,0.058820,3.463272
min,0.000000,2020-01-01 00:00:00,23.000000,0.000000,0.000000,0.000000,0.000000,1.003200e+05,-491.000000,1031.000000,5703.000000,136.000000,2.000000,0.007305,0.002311,0.015513
25%,256.000000,2020-09-13 00:00:00,2359.500000,117.000000,518.500000,0.000000,4.000000,7.044475e+05,1051.500000,13424.500000,43383.500000,1652.000000,216.000000,0.063815,0.026271,0.862109
50%,501.000000,2021-05-16 00:00:00,5050.000000,230.000000,1000.000000,11323.000000,7.000000,1.285463e+06,2294.000000,25253.000000,62502.000000,2772.000000,475.000000,0.102687,0.044779,1.800858
75%,748.500000,2022-01-18 12:00:00,7387.000000,361.500000,1544.000000,29012.000000,11.000000,1.794376e+06,3633.500000,38039.500000,79777.500000,3897.500000,736.500000,0.149810,0.067282,3.515925
max,999.000000,2022-09-26 00:00:00,9988.000000,499.000000,1999.000000,49877.000000,14.000000,2.357788e+06,4998.000000,49901.000000,99974.000000,4998.000000,999.000000,1.356033,0.840961,78.564500
std,286.520093,NaN,2855.201310,144.088562,582.733167,15896.760567,4.108038,6.403143e+05,1535.226096,14201.982311,23107.211844,1338.671140,291.586571,0.127840,0.063477,6.124132


In [10]:
df.to_csv("social_media_data.csv", index = False)